In [1]:
from typing import Tuple, List, Dict
from itertools import chain
import json
import requests
from copy import deepcopy
from itertools import permutations, groupby
import csv
from dataclasses import dataclass
from statistics import median
from collections import defaultdict, Counter

In [2]:
def load_results(meta_dict: Dict) -> List:
    url = f'https://rating.chgk.info/api/tournaments/{meta_dict["id"]}/list.json'
    resp = requests.get(url).json()
    for team in resp:
        team['meta'] = meta_dict
    return resp

In [3]:
class Team:
    def __init__(self, dicts: List[Dict]):
        self.name = dicts[0]['base_name']
        self.id = dicts[0]['idteam']
        self.results = self.parse_results(dicts)
        self.starting_group = dicts[0]['meta']['group']
        self.rating = int(dicts[0]['tech_rating_rg'])
        self.sum = 0
        self.sums = []
        self.place = None
        
    def __repr__(self):
        return json.dumps(self.__dict__, ensure_ascii=False)
    
    @staticmethod
    def parse_results(stages: List[Dict]):
        zipped = [dict(zip(stage['meta']['editors'], Team.parse_mask(stage['mask'])))
                   for stage in stages]
        return {**zipped[0], **zipped[1], **zipped[2], **zipped[3]}
    
    @staticmethod
    def parse_mask(mask: str) -> Tuple[int, int]:
        first_tour, second_tour = mask[:15], mask[15:]
        return first_tour.count('1'), second_tour.count('1')
    
    @property
    def previous_sum(self):
        return sum(self.sums[:-1])
    
    @property
    def current_sum(self):
        return self.sums[-1] if self.sums else 0
            

In [4]:
def generate_tournaments(editors):
    perms = permutations(editors)
    paired_perms = ((p[0:2], p[2:4], p[4:6], p[6:8]) for p in perms)
    sorted_pairs = (tuple(tuple(sorted(pair)) for pair in perm) for perm in paired_perms)
    return set(sorted_pairs)

In [5]:
@dataclass
class TeamResult:
    name: str
    place: int
    editors: str
    final_sum: int
    final_editors: str
    actual_place: int = None

def run(tournament, a, b, c):
    for stage in range(3):
        run_stage(tournament[stage], a, b, c)
        a, b, c = move_teams(*sort_preliminary_stage(a, b, c))

    run_stage(tournament[3], a, b, c)
    all_teams = place_teams(a, b, c)
    editors = ', '.join(' и '.join(e for e in pair) for pair in tournament)
    final_editors = ' и '.join(tournament[3])
    return [TeamResult(team.name, team.place, editors, team.current_sum, final_editors)
            for team in all_teams]
        
        
def run_stage(editors_pair, a, b, c):
    for team in [*a, *b, *c]:
        team.sums.append(team.results[editors_pair[0]] + team.results[editors_pair[1]])
    
    
def sorting_key(team: Team):
    return (team.current_sum, team.previous_sum, -team.rating)


def sort_preliminary_stage(a, b, c):
    return (sorted(a, key=sorting_key, reverse=True),
            sorted(b, key=sorting_key, reverse=True),
            sorted(c, key=sorting_key, reverse=True))


def place_teams(a, b, c):
    sorted_final = sort_final(a)
    
    sorted_non_final = (*sorted(b, key=sorting_key, reverse=True),
            *sorted(c, key=sorting_key, reverse=True))
    
    placed = zip(range(len(sorted_final) + 1, len(a) + len(b) + len(c) + 1), sorted_non_final)
    for place, team in placed:
        team.place = place

    return (*sorted_final, *sorted_non_final)


def sort_final(teams):
    first, second, third = sorted((t.current_sum for t in teams), reverse=True)[:3]
                 
    for t in teams:
        if t.current_sum == first:
            t.place = 1
        elif t.current_sum == second and second < first:
            t.place = 2
        elif t.current_sum == third and third < second:
            t.place = 3
                 
    not_placed = sorted((team for team in teams if team.place is None), 
                        key=sorting_key, reverse=True)
    for index, team in enumerate(not_placed, len(teams) - len(not_placed) + 1):
        team.place = index
                 
    return sorted(teams, key=lambda t: t.place)


def move_teams(a, b, c):
    move_a_b = round((len(a) + 0.1) / 4)
    move_b_c = round((len(b) + 0.1) / 4)
    new_a = [*a[:len(a) - move_a_b], *b[:move_a_b]]
    new_b = [*a[len(a) - move_a_b:], *b[move_a_b:len(b) - move_b_c], *c[:move_b_c]]
    new_c = [*b[len(b) - move_b_c:], *c[move_b_c:]]
    return new_a, new_b, new_c

In [6]:
def load_and_parse_rating_data(metadata) -> List[Team]:
    raw_results = (load_results(dict_) for dict_ in metadata)
    flat_results = list(chain.from_iterable(raw_results))
    team_ids = set(result['idteam'] for result in flat_results)
    team_dicts = [[dict_ for dict_ in flat_results if dict_['idteam'] == team_id] 
                  for team_id in team_ids]
    return [Team(dicts) for dicts in team_dicts]


def run_simulations(metadata, teams) -> List[TeamResult]:
    group_a = [t for t in teams if t.starting_group == 'A']
    group_b = [t for t in teams if t.starting_group == 'B']
    group_c = [t for t in teams if t.starting_group == 'C']
    
    editors = sorted(chain.from_iterable(set(tour['editors'] for tour in metadata)))
    tournaments = generate_tournaments(editors)
    
    
    return list(chain.from_iterable(run(t, 
                                       deepcopy(group_a), 
                                       deepcopy(group_b), 
                                       deepcopy(group_c)) 
                               for t in tournaments))


def get_actual_results(metadata, all_results: List[TeamResult]) -> Dict:
    actual_editors = ', '.join(' и '.join(e for e in pair) 
                          for pair in [sorted(metadata[i]['editors']) 
                                       for i in (0, 3, 6, 9)])
    return {r.name: r.place for r in all_results
            if r.editors == actual_editors}

In [7]:
def export_to_csv(all_results: List[TeamResult], filename: str):
    with open(filename, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(("Команда", 
                         "Команда", 
                         "Место", 
                         "Редакторы",
                         "Редакторы финала",
                         "Сумма в финале",
                         "Настоящее место",))
        for team in all_results:
            writer.writerow((team.name, 
                             f"{team.actual_place}. {team.name}", 
                             team.place, 
                             team.editors,
                             team.final_editors, 
                             team.final_sum, 
                             team.actual_place))

def export_sum_to_csv(teams: List[Team], filename: str):
    with open(filename, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(("Команда", "Сумма"))
        for team in teams:
            writer.writerow((team.name, sum(team.results.values())))
            

def export_stats_to_csv(all_results: List[TeamResult], filename: str):
    single_team_results = []
    sorted_results = sorted(all_results, key=lambda tr: tr.name)
    for _, group in groupby(sorted_results, key=lambda tr: tr.name):
        single_team_results.append(list(group))
        
    with open(filename, 'w') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(("Команда", 
                         "Настоящее место",
                        "Медиана",
                        "Мода",
                        "Лучшее место",
                        "Худшее место",))
        for team in single_team_results:
            mode = Counter(tr.place for tr in team).most_common()[0][0]
            writer.writerow((team[0].name,
                           team[0].actual_place,
                           int(median(tr.place for tr in team)),
                           mode,
                           min(tr.place for tr in team),
                           max(tr.place for tr in team),))

In [8]:
def run_year(metadata, year: int):
    teams = load_and_parse_rating_data(metadata)
    export_sum_to_csv(teams, f'{year}_sum.csv')
    
    all_results = run_simulations(metadata, teams)
    
    actual_results = get_actual_results(metadata, all_results)
    for r in all_results:
        r.actual_place = actual_results[r.name]
        
    export_to_csv(all_results, f'{year}.csv')
    export_stats_to_csv(all_results, f'{year}_stats.csv')

In [9]:
metadata_2018 = [
    {'editors': ('Колмаков', 'Саксонов'), 'id': 5244, 'group': 'A'},
    {'editors': ('Колмаков', 'Саксонов'), 'id': 5245, 'group': 'B'},
    {'editors': ('Колмаков', 'Саксонов'), 'id': 5246, 'group': 'C'},
    {'editors': ('Иванов', 'Кудрявцев'), 'id': 5252, 'group': 'A'},
    {'editors': ('Иванов', 'Кудрявцев'), 'id': 5253, 'group': 'B'},
    {'editors': ('Иванов', 'Кудрявцев'), 'id': 5254, 'group': 'C'},
    {'editors': ('Абрамов', 'Терентьев'), 'id': 5255, 'group': 'A'},
    {'editors': ('Абрамов', 'Терентьев'), 'id': 5256, 'group': 'B'},
    {'editors': ('Абрамов', 'Терентьев'), 'id': 5257, 'group': 'C'},
    {'editors': ('Скиренко', 'Карпук'), 'id': 5259, 'group': 'A'},
    {'editors': ('Скиренко', 'Карпук'), 'id': 5260, 'group': 'B'},
    {'editors': ('Скиренко', 'Карпук'), 'id': 5261, 'group': 'C'},
]

In [10]:
metadata_2017 = [
    {'editors': ('Колмаков', 'Иванов'), 'id': 4615, 'group': 'A'},
    {'editors': ('Колмаков', 'Иванов'), 'id': 4616, 'group': 'B'},
    {'editors': ('Колмаков', 'Иванов'), 'id': 4617, 'group': 'C'},
    {'editors': ('Мерзляков', 'Кудрявцев'), 'id': 4618, 'group': 'A'},
    {'editors': ('Мерзляков', 'Кудрявцев'), 'id': 4619, 'group': 'B'},
    {'editors': ('Мерзляков', 'Кудрявцев'), 'id': 4620, 'group': 'C'},
    {'editors': ('Ершов', 'Коробейников'), 'id': 4621, 'group': 'A'},
    {'editors': ('Ершов', 'Коробейников'), 'id': 4622, 'group': 'B'},
    {'editors': ('Ершов', 'Коробейников'), 'id': 4623, 'group': 'C'},
    {'editors': ('Абрамов', 'Сборный тур'), 'id': 4624, 'group': 'A'},
    {'editors': ('Абрамов', 'Сборный тур'), 'id': 4625, 'group': 'B'},
    {'editors': ('Абрамов', 'Сборный тур'), 'id': 4626, 'group': 'C'},
]

In [11]:
run_year(metadata_2018, 2018)

In [12]:
run_year(metadata_2017, 2017)